### Setup and imports
Let us first get our imports out of the way:

In [1]:
from pydrake.all import *

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from manipulation.meshcat_utils import AddMeshcatTriad

from pathlib import Path
import numpy as np
import sys

# Add src to path for imports
sys.path.append(str(Path.cwd()))
from throw_helpers import (
    interpolate_poses_linear,
    interpolate_poses_arc_motion,
    create_q_knots,
)

### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [3]:
abs_table_sdf_path = f"{Path.cwd()}/assets/table.sdf"
with open(abs_table_sdf_path, "r") as fin:
    table_sdf = fin.read()

scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{abs_table_sdf_path}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}

- add_model:
    name: ball0
    file: package://drake_models/manipulation_station/sphere.sdf
    default_free_body_pose:
        base_link:
            translation: [0, -0.5, 0.02]
            rotation: !Rpy {{ deg: [0, 0, 0] }}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

- Modify mass of ball?
- Add more points along the arc
- Try just flinging one joint angle (with more resolution)

### Pick and Place Functions

Helper functions for designing grasp poses and trajectories.

In [4]:
def design_grasp_pose(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """Design a grasp pose 0.17m above the object."""
    p_OG = [0, 0, 0.12]
    R_OG = RotationMatrix.MakeXRotation(-np.pi/2)
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_OG, X_WG

def approach_pose(X_WG: RigidTransform) -> RigidTransform:
    """Create an approach pose 0.1m above the grasp pose."""
    X_WGApproach = RigidTransform(p=[0, 0, 0.1]) @ X_WG
    return X_WGApproach

def make_trajectory(
    X_Gs: list[RigidTransform], finger_values: np.ndarray, sample_times: list[float]
) -> tuple[Trajectory, PiecewisePolynomial]:
    """Convert keyframes to velocity trajectory and gripper commands."""
    piecewise = PiecewisePose.MakeLinear(sample_times, X_Gs)
    robot_velocity_trajectory = piecewise.MakeDerivative()
    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(sample_times, finger_values)
    return robot_velocity_trajectory, traj_wsg_command

### Jacobian-Based Velocity Controller

In [5]:
class PoseTrajectorySource(LeafSystem):
    """System that outputs poses from a PiecewisePose trajectory."""
    def __init__(self, trajectory: PiecewisePose):
        LeafSystem.__init__(self)
        self._trajectory = trajectory
        self.DeclareAbstractOutputPort(
            "pose",
            lambda: AbstractValue.Make(RigidTransform()),
            self.CalcPose
        )
    
    def CalcPose(self, context: Context, output):
        output.set_value(self._trajectory.GetPose(context.get_time()))

class PoseTrajectoryController(LeafSystem):
    """Controller that tracks a pose trajectory using differential IK with feedback."""
    def __init__(self, plant: MultibodyPlant, kp: float = 5.0):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.X_WG_desired_port = self.DeclareAbstractInputPort("X_WG_desired", AbstractValue.Make(RigidTransform()))
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()
        
        # Gain for position error feedback (increased from 2.0 to 5.0)
        self.kp = kp

    def CalcOutput(self, context: Context, output: BasicVector):
        X_WG_desired = self.X_WG_desired_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        
        # Current pose
        X_WG = self._plant.CalcRelativeTransform(self._plant_context, self._W, self._G)
        
        # Pose error in spatial velocity form
        X_err = X_WG_desired.multiply(X_WG.inverse())
        angle_axis = X_err.rotation().ToAngleAxis()
        p_err = X_err.translation()
        w_err = angle_axis.angle() * angle_axis.axis()
        V_err = np.hstack([w_err, p_err])
        
        # Compute Jacobian
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],
            self._W,
            self._W
        )
        J_G = J_G[:, self.iiwa_start : self.iiwa_end + 1]
        
        # Velocity command with proportional feedback on pose error
        v = np.linalg.pinv(J_G) @ (self.kp * V_err)
        output.SetFromVector(v)

### Build Diagram with Pick and Place Controller

In [6]:
# Build the diagram with controller
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")
scenegraph = station.GetSubsystemByName("scene_graph")

# Clear previous meshcat objects
meshcat.Delete()

# Get initial poses
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))

# Get ball pose
ball_body = plant.GetBodyByName("base_link", plant.GetModelInstanceByName("ball0"))
X_WO_ball = plant.EvalBodyPoseInWorld(temp_plant_context, ball_body)

# Design grasp and approach poses
X_OG, X_WG_pick = design_grasp_pose(X_WO_ball)
X_WG_prepick = approach_pose(X_WG_pick)

# Visualize pick and prepick poses with RGB triads
# AddMeshcatTriad(meshcat, "prepick_pose", length=0.15, radius=0.005, X_PT=X_WG_prepick)
# AddMeshcatTriad(meshcat, "pick_pose", length=0.15, radius=0.005, X_PT=X_WG_pick)

# Add frame triads for gripper and ball bodies
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName("body"),
    length=0.15,
    radius=0.005
)
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=ball_body,
    length=0.1,
    radius=0.003
)

# Gripper states
opened = 0.107
closed = 0.0

In [7]:
# Arc-based throwing motion with IK trajectory

# Timing parameters (easily adjustable)
t_pickup_go = 2.0
t_pickup_hold = 0.5
t_pickup_return = 2.0
t_throw_windup = 1.5
t_throw_arc = 0.4  # Fast arc motion for throwing
t_throw_followthrough = 0.3

# Target location (hardcoded)
p_world_target = np.array([0.5, -1.2, 0.5])  # Forward and up
AddMeshcatTriad(meshcat, "target", length=0.2, radius=0.01, 
                X_PT=RigidTransform(p=p_world_target))

# === PICKUP PHASE ===
# Generate dense trajectory for pickup
t_pickup_total = t_pickup_go + t_pickup_hold + t_pickup_return
num_pickup_samples = 100
t_pickup_lst = np.linspace(0, t_pickup_total, num_pickup_samples, endpoint=False)
pose_pickup_lst = []

for t in t_pickup_lst:
    if t < t_pickup_go:
        # Go to prepick, then pick
        progress = t / t_pickup_go
        if progress < 0.5:
            pose = interpolate_poses_linear(X_WGinitial, X_WG_prepick, progress * 2)
        else:
            pose = interpolate_poses_linear(X_WG_prepick, X_WG_pick, (progress - 0.5) * 2)
    elif t < t_pickup_go + t_pickup_hold:
        # Hold at pick position
        pose = X_WG_pick
    else:
        # Return to initial
        progress = (t - t_pickup_go - t_pickup_hold) / t_pickup_return
        pose = interpolate_poses_linear(X_WG_pick, X_WGinitial, progress)
    
    pose_pickup_lst.append(pose)

# === THROWING PHASE ===
# Define throw poses based on target direction
theta = np.arctan2(p_world_target[1], p_world_target[0])  # Direction to target

# Pre-throw pose: positioned to wind up
prethrow_radius = 0.4
prethrow_height = 0.3
T_world_prethrow = RigidTransform(
    p=np.array([
        prethrow_radius * np.cos(theta),
        prethrow_radius * np.sin(theta),
        prethrow_height
    ]),
    R=RotationMatrix.MakeXRotation(-np.pi/2).multiply(
        RotationMatrix.MakeYRotation(theta - np.pi/2)
    )
)

# Throw pose: higher and forward for release
throw_height = 0.5
throw_radius_offset = 0.3
T_world_throw = RigidTransform(
    p=T_world_prethrow.translation() + np.array([
        throw_radius_offset * np.cos(theta),
        throw_radius_offset * np.sin(theta),
        throw_height - prethrow_height
    ]),
    R=RotationMatrix.MakeXRotation(-np.pi/2).multiply(
        RotationMatrix.MakeYRotation(theta - np.pi/2).multiply(
            RotationMatrix.MakeXRotation(-np.pi/2)
        )
    )
)

# Visualize throw poses
AddMeshcatTriad(meshcat, "prethrow", length=0.15, radius=0.005, X_PT=T_world_prethrow)
AddMeshcatTriad(meshcat, "throw", length=0.15, radius=0.005, X_PT=T_world_throw)

# Generate dense trajectory for throwing
num_throw_samples = 150
t_throw_total = t_throw_windup + t_throw_arc + t_throw_followthrough
t_throw_lst = np.linspace(0, t_throw_total, num_throw_samples, endpoint=False)
pose_throw_lst = []

for t in t_throw_lst:
    if t < t_throw_windup:
        # Wind up: go from initial to prethrow
        progress = t / t_throw_windup
        pose = interpolate_poses_linear(X_WGinitial, T_world_prethrow, progress)
    elif t < t_throw_windup + t_throw_arc:
        # Arc motion for throw
        progress = (t - t_throw_windup) / t_throw_arc
        pose = interpolate_poses_arc_motion(T_world_prethrow, T_world_throw, progress)
    else:
        # Follow through: continue arc motion slightly
        pose = T_world_throw
    
    pose_throw_lst.append(pose)

# Combine all poses
all_poses = pose_pickup_lst + pose_throw_lst

# Convert poses to joint angles using IK
print(f"Converting {len(all_poses)} poses to joint angles using IK...")
q_knots = create_q_knots(plant, all_poses)
print(f"Successfully computed {len(q_knots)} joint configurations")

# Create time arrays
t_pickup_array = t_pickup_lst
t_throw_array = t_throw_lst + t_pickup_total
t_all = np.concatenate([t_pickup_array, t_throw_array])

# Create joint trajectory
q_knots_array = np.array(q_knots).T  # Shape: (7, num_knots)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_all[:len(q_knots)], q_knots_array)

# Create gripper trajectory
# Gripper opens at release (during arc motion)
t_release = t_pickup_total + t_throw_windup + t_throw_arc * 0.7  # Release 70% through arc
gripper_times = np.array([
    0.0,
    t_pickup_go,
    t_pickup_go + t_pickup_hold,
    t_pickup_total,
    t_pickup_total + t_throw_windup,
    t_release,
    t_all[len(q_knots)-1]
])
gripper_positions = np.array([
    opened,  # Start open
    opened,  # Still open at pick
    closed,  # Close after hold
    closed,  # Closed during return
    closed,  # Closed during windup
    opened,  # Open at release
    opened   # Stay open
]).reshape(1, -1)
traj_wsg_command = PiecewisePolynomial.FirstOrderHold(gripper_times, gripper_positions)

print(f"Total trajectory time: {t_all[len(q_knots)-1]:.2f} seconds")

Converting 250 poses to joint angles using IK...
IK failed at pose 224/250: IK failed to find solution for pose: RigidTransform(
  R=RotationMatrix([
    [-0.9230769230769231, 0.3651636452468864, -0.12076632921823316],
    [0.384615384615385, 0.8763927485925262, -0.2898391901237592],
    [1.3877787807814457e-17, -0.3139924559674058, -0.9494254776419038],
  ]),
  p=[0.2268788828955311, -0.5445093189492746, 0.43720150880651887],
)
Successfully computed 224 joint configurations
Total trajectory time: 6.30 seconds


In [9]:
# Build diagram with position control
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)

# Add trajectory sources
q_source = builder.AddSystem(TrajectorySource(q_traj))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connect to station
builder.Connect(q_source.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()

# Run simulation
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
diagram.ForcedPublish(context)

print(f"Running simulation for {q_traj.end_time():.2f} seconds")
meshcat.StartRecording()
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(q_traj.end_time() + 5)
meshcat.StopRecording()
meshcat.PublishRecording()

Running simulation for 6.30 seconds
